#### Trick-or-treat.ipynb: going house to house in each neighborhood

In [2]:
import pandas as pd
import os
from lxml import html
import requests
import time

In [3]:
df = pd.read_csv('./data-06-2023/texas/austin-central-texas.csv')

In [4]:
df.shape

(556, 8)

In [5]:
def cleanURL(url):
    """
    Takes lennar url as in all the csv files and returns the updated version ready to scrape
    returns: String (url)
    """
    
    ## get rid of code if the last part of url starts with a digit
    if url.split('/')[-1][0].isdigit():
        tmp = url.split('/')[:-1]
        url = '/'.join(tmp)
    
    final_url = url + '/included?print=true'
    return final_url

In [8]:
csv_files = []
path = './data-06-2023/'
for root, dirs, files in os.walk(path):
    for name in files:
        if name.endswith('.csv'):
            csv_files.append(root+'/'+name)

In [20]:
csv_files[67:]

['./data-06-2023/california/palm-springs-coachella.csv',
 './data-06-2023/california/san-francisco-metro-area.csv',
 './data-06-2023/california/.ipynb_checkpoints/palm-springs-coachella-checkpoint.csv',
 './data-06-2023/california/.ipynb_checkpoints/la-orange-county-checkpoint.csv']

In [21]:
## main loop
for f in csv_files[67:]:
    df = pd.read_csv(f)
    target = df.shape[0]
    print(f)
    print(target)
    
    data_dump = [] # list which will be one column in the new df
    new_rows = [] # list of lists representing rows for each home of different length with each attribute
    
    for u in df['URL']:
        newURL = cleanURL(u)
        time.sleep(0.05)
        print('.', end='')
        
        try:
            # print(newURL)
            page = requests.get(newURL)
            tree = html.fromstring(page.content)  
            index = 3
            keep_scraping = True
            d = dict()
            
            ## if page isnt found and it's redirected, continue
            if page.status_code == 200 and 'redirectNotice=true' in page.url:
                data_dump.append(d)
                new_rows.append([])
                continue
                
            ## or it's not found
            if page.status_code == 200 and 'included' not in page.url:
                data_dump.append(d)
                new_rows.append([])
                continue
            
            else:
                while keep_scraping:
                    try:
                        key = tree.xpath(f'//*[@id="main"]/div/div/div/div[{index}]/div[1]/text()')[0]
                        val = tree.xpath(f'//*[@id="main"]/div/div/div/div[{index}]/div[2]/*/text()')

                        index += 1
                        d[key] = val
                    except IndexError:
                        del d[key] # remove the last key we looked at which is the extra info at bottom of page
                        # print(d)
                        keep_scraping = False

                data_dump.append(d)
                new_rows.append([item for sublist in d.values() for item in sublist])
                
        except: 
            print("Something is wrong with connecting to the webpage -- maybe with URL")
            break
    
    assert target == len(data_dump) and target == len(new_rows)
    
    ## add collected dataframe back
    df['Data_Dump'] = data_dump
    tmp = pd.DataFrame(new_rows)
    tmp.fillna("",inplace=True)
    tmp = tmp.add_prefix('feature_')
    
    ## combine
    final = pd.concat([df, tmp], axis=1)
    
    ## resave
    path = '.'+f.split('.')[1]+'-final.json'
    final.to_json(path)

./data-06-2023/california/palm-springs-coachella.csv
37
....................................../data-06-2023/california/san-francisco-metro-area.csv
12
............./data-06-2023/california/.ipynb_checkpoints/palm-springs-coachella-checkpoint.csv
37
....................................../data-06-2023/california/.ipynb_checkpoints/la-orange-county-checkpoint.csv
221
.............................................................................................................................................................................................................................

In [86]:
# final.to_json('./data/north-carolina/charlotte-finalvan.json')

In [87]:
# pd.read_json('./data/north-carolina/charlotte-final.json')

In [106]:
data_dump[44]['Connectivity']

['Wifi heat mapping (engineering)',
 'Legrand On-Q 42 inch RF Transparent Structured Media&reg; Enclosure',
 'Ring Alarm Security Kit &ndash; Base Station**',
 'Ring Alarm Security Kit &ndash; Ring Alarm Keypad*',
 'Ring Alarm Security Kit &ndash; Ring Alarm Motion Detector**',
 'Ring Alarm Security Kit &ndash; Ring Alarm Smoke and CO Listener**',
 'Ring Alarm Security Kit &ndash; Ring Alarm Window and Door Contact Sensor**',
 'Ring Video Doorbell Pro',
 'Honeywell Home T6 Pro Z-Wave - smart thermostat',
 'Schlage Encode Smart WiFi Deadbolt']